In [1]:
# Copyright 2014 Brett Slatkin, Pearson Education Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value

In [3]:
data = LazyDB()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)

Before: {'exists': 5}
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


In [4]:
# __getattr_ runs once to do hard work of loading property
class LoggingLazyDB(LazyDB):
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)

data = LoggingLazyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

exists: 5
Called __getattr__(foo)
foo:    Value for foo
foo:    Value for foo


In [5]:
# __getattribute__ is called every time an attribute is accessed on an object
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value

data = ValidatingDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

Called __getattribute__(exists)
exists: 5
Called __getattribute__(foo)
foo:    Value for foo
Called __getattribute__(foo)
foo:    Value for foo


In [6]:
# AttributeError for missing property behavior
import logging
try:
    class MissingPropertyDB(object):
        def __getattr__(self, name):
            if name == 'bad_name':
                raise AttributeError('%s is missing' % name)
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value

    data = MissingPropertyDB()
    data.foo  # Test this works
    data.bad_name
except:
    logging.exception('Expected')
else:
    assert False

In [7]:
# hasattr
data = LoggingLazyDB()
print('Before:     ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))
print('After:      ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))

Before:      {'exists': 5}
Called __getattr__(foo)
foo exists:  True
After:       {'exists': 5, 'foo': 'Value for foo'}
foo exists:  True


In [8]:
# __getattribute__ called each time hasattr or getattr is accessed on an object
data = ValidatingDB()
print('foo exists: ', hasattr(data, 'foo'))
print('foo exists: ', hasattr(data, 'foo'))

Called __getattribute__(foo)
foo exists:  True
Called __getattribute__(foo)
foo exists:  True


In [9]:
class SavingDB(object):
    def __setattr__(self, name, value):
        # Save some data to the DB log
        super().__setattr__(name, value)

In [10]:
# __setattr__ is called everytime 
class LoggingSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print('Called __setattr__(%s, %r)' % (name, value))
        super().__setattr__(name, value)

data = LoggingSavingDB()
print('Before: ', data.__dict__)
data.foo = 5
print('After:  ', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)

Before:  {}
Called __setattr__(foo, 5)
After:   {'foo': 5}
Called __setattr__(foo, 7)
Finally: {'foo': 7}


In [11]:
class BrokenDictionaryDB(object):
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        return self._data[name]

In [13]:
# __getattribute__ access self._data which causes __getattribute__ to run again
try:
    data = BrokenDictionaryDB({'foo': 3})
    data.foo
except:
    logging.exception('Expected')
else:
    assert False

Called __getattribute__(foo)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __g

In [14]:
# Use the super().__getattribute__ to avoid recursion
class DictionaryDB(object):
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

data = DictionaryDB({'foo': 3})
print(data.foo)

3
